### Convert x,y coordinates from the laser to x,y coordinates of the mount map image using petro-image
##### By Glenn R. Sharman for use by the Clastic Stratigraphy Research Group, University of Arkansas
##### Updated May  2, 2025

In [ ]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
import pandas as pd
import random
from PIL import Image
import transformFuncs as tFunc

from importlib import reload # For testing

##### To use this code, you will need the following: 1) the image of the mount, 2) the image coordinates of 6 or more (preferrably 10+) of the spot locations on the image as a geojson file (exported from petro-image), and 3) a CSV file with the laser coordinates of all the spots (typically provided for the entire sequence).

##### You will also need to specify the output filepaths for 1) the image of the mount map with transformed spot locations, and 2) the geojson with x,y coordinates of the spot locations.

##### Note: <i>The algorithm uses a quadratic fit and will provide an approximation of the laser spot locations on the mount image. The transformation will not likely be perfect. Always confirm spot locations visually. </i>

##### Note: <i>The labels in petro-image should be numbers only</i>

In [ ]:
# Required information

# The name of the sample, which must match what is in the laser coordinates CSV file
sample_id = 'S-FWS-24-01'

# The geojson with image coordinates of selected spots
input_geojson = 'example-data/FSW24-01_zr_sticky.geojson'

# The pathway to the mount map image file
image_path = 'example-data/FWS24-01 5x Zr sticky tape mount.jpg'

# The pathway to the Excel file with laser image coordinates
laser_coords_file = 'example-data/J250108_03_Spots.csv'

# The pathway of the jpg that will be exported
image_output = 'outputs/FWS24-01_zr_sticky tape mount with spots.jpg'

# The pathway of the GeoJSON that will be exported
json_output = 'outputs/FWS24-01_zr_sticky_wSpots.geojson'

# Proportion of y-axis range to adjust the label so that it sits a bit up and to the right of the spot
adjust = 0.005

# Output image resolution
dpi = 300

# Specify whether to plot points as circles
plot_circle = True # True or False
diameter_um = 25 # Diameter of the circle, in microns

##### Execute the code

In [ ]:
# Open and load the GeoJSON file
with open(input_geojson, 'r') as file:
    data = json.load(file)  # Parses JSON into a Python dictionary
df = pd.DataFrame(data['features'])

# Load x and y coordinates and labels into lists
x_px = []
y_px = []
labels = []
for i in range(len(df)):
    x_px.append(df.iloc[i]['geometry']['coordinates'][0])
    y_px.append(df.iloc[i]['geometry']['coordinates'][1])
    labels.append(df.iloc[i]['properties']['label'])

# Open the CSV with the laser coordinates
laser_coords_all = pd.read_csv(laser_coords_file)

# Split the label into two new columns: 'Sample_ID' and 'Analysis_ID'
laser_coords_all[['Sample_ID', 'Analysis_ID']] = laser_coords_all['Name'].str.split('.', expand=True)

# Select only the sample that is of interest
laser_coords_all = laser_coords_all[laser_coords_all['Sample_ID'] == sample_id]

# Convert Analysis_ID to text
laser_coords_all['Analysis_ID'] = laser_coords_all['Analysis_ID'].astype(str)

# Select only the laser coordinates that have matching image coodinates
laser_coords = laser_coords_all[laser_coords_all['Analysis_ID'].isin(labels)]

# Open and load the image
Image.MAX_IMAGE_PIXELS = 500000000 # To avoid errors that the files are too large
image = Image.open(image_path)

# Set A, B, and C variable for 

# image coordinates
A = np.array([df.iloc[x]['geometry']['coordinates'] for x in range(len(df))])
# laser coordinates
B = np.array(list(zip(laser_coords['X(um)'],laser_coords['Y(um)'])))
# laser coordinates (all)
C = np.array(list(zip(laser_coords_all['X(um)'],laser_coords_all['Y(um)'])))

# Compute Quadratic Transformation
coeff_x, coeff_y = tFunc.compute_quadratic_transform(B, A)

# Apply Transformation
C_transformed = tFunc.apply_quadratic_transform(C, coeff_x, coeff_y)

# Prepare GeoJSON structure
geojson = {
    "type": "FeatureCollection",
    "features": []  # Initialize as an empty list to store features
}

# Note: The properties of the first annotation in the input GeoJSON will be mimicked
pixels_per_meter = df.iloc[0]['properties']['pixelsPerMeter']
r = diameter_um/2 * pixels_per_meter / 1e6
num_vertices = 100  # number of points around the circle

# Export a GeoJSON with the results
if plot_circle:
    for i in range(len(C_transformed)):
        # x,y coordinates of the circle
        angles = np.linspace(0, 2 * np.pi, num_vertices + 1)
        x_coords = C_transformed[i][0] + r * np.cos(angles)
        y_coords = C_transformed[i][1] + r * np.sin(angles)
        
        feature = {
                        "type": "Feature",
                        "geometry": {
                            "type": 'Polygon',
                            "coordinates": list([list(zip(x_coords, y_coords))]) # coordinates_array  # Should wrap in an extra list for GeoJSON Polygon
                        },
                        "properties": {
                            "uuid": tFunc.generate_unique_id(16),
                            "label": laser_coords_all.iloc[i]['Analysis_ID'],
                            "xLabel": C_transformed[i][0],
                            "yLabel": C_transformed[i][1],
                            "imageTitle": df.iloc[0]['properties']['imageTitle'],
                            "pixelsPerMeter": pixels_per_meter,
                            "imageWidth": df.iloc[0]['properties']['imageWidth'],
                            "imageHeight": df.iloc[0]['properties']['imageHeight'],
                            "labelFontSize": df.iloc[0]['properties']['labelFontSize'],
                            "labelFontColor": df.iloc[0]['properties']['labelFontColor'],
                            "labelBackgroundColor": df.iloc[0]['properties']['labelBackgroundColor'],
                            "labelBackgroundOpacity": df.iloc[0]['properties']['labelBackgroundOpacity'],
                            "lineWeight": df.iloc[0]['properties']['lineWeight'],
                            "lineColor": df.iloc[0]['properties']['lineColor'],
                            "lineOpacity": df.iloc[0]['properties']['lineOpacity'],
                            "fillColor": '#000000',
                            "fillOpacity": 0
                        }
                    }
        geojson["features"].append(feature)
    with open(json_output, "w") as geojson_file:
        json.dump(geojson, geojson_file, indent=2)
else:
    for i in range(len(C_transformed)):
        feature = {
                        "type": "Feature",
                        "geometry": {
                            "type": 'Point',
                            "coordinates": list(C_transformed[i]) # coordinates_array  # Should wrap in an extra list for GeoJSON Polygon
                        },
                        "properties": {
                            "uuid": tFunc.generate_unique_id(16),
                            "label": laser_coords_all.iloc[i]['Analysis_ID'],
                            "xLabel": C_transformed[i][0],
                            "yLabel": C_transformed[i][1],
                            "imageTitle": df.iloc[0]['properties']['imageTitle'],
                            "pixelsPerMeter": pixels_per_meter,
                            "imageWidth": df.iloc[0]['properties']['imageWidth'],
                            "imageHeight": df.iloc[0]['properties']['imageHeight'],
                            "labelFontSize": df.iloc[0]['properties']['labelFontSize'],
                            "labelFontColor": df.iloc[0]['properties']['labelFontColor'],
                            "labelBackgroundColor": df.iloc[0]['properties']['labelBackgroundColor'],
                            "labelBackgroundOpacity": df.iloc[0]['properties']['labelBackgroundOpacity'],
                            "lineWeight": df.iloc[0]['properties']['lineWeight'],
                            "lineColor": df.iloc[0]['properties']['lineColor'],
                            "lineOpacity": df.iloc[0]['properties']['lineOpacity']
                        }
                    }
        geojson["features"].append(feature)
    with open(json_output, "w") as geojson_file:
        json.dump(geojson, geojson_file, indent=2)

# Make and export the figure
fig, ax = plt.subplots(1, figsize=(image.size[0]/dpi,image.size
[1]/dpi))

labels = [tFunc.get_digits_after_last_period(laser_coords_all.iloc[x]['Name']) for x in range(len(laser_coords_all))]

adjust_px = adjust*image.size[1]

#ax.plot(x_px, y_px, '+', label='petro-image coordinates', color='navy')
if plot_circle:
    for i in range(len(C_transformed)):
        circle = patches.Circle((C_transformed[i][0], C_transformed[i][1]), r, edgecolor='red', facecolor='none', linewidth=2)
        ax.add_patch(circle)    
else:
    ax.plot([x[0] for x in C_transformed], [x[1] for x in C_transformed], '+', label='laser coordinates', color='red')
for i in range(len(C_transformed)):
    ax.text(x=C_transformed[i][0]+adjust_px, y=C_transformed[i][1]-adjust_px, s=labels[i], fontsize='medium', color='red',)
ax.imshow(image)
ax.legend()
plt.axis('off')

fig.savefig(image_output, dpi=dpi)

In [ ]:
np.asarray([list(zip(x_coords, y_coords))])

In [ ]:
x_coords